In [ ]:
import torch
import copy
import numpy as np
import pandas as pd  

from torch import nn
from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from algorithms.decision_tree import DecisionTree

from models.test_models import ProphecyPaperNetwork, TestModel
from models.acasxu_1_1 import Acasxu1_1
from models.utils import attach_relu_activation_hook, attach_layer_output_hook, get_layers_info, turn_bool_activation_to_int

### Labeled dataset for Prophecy paper's model

In [ ]:
# we'll consider a range of -50 to 50 for our inputs x1 and x2. 
# We'll iterate through all possible pairs of (x1, x2) within this range, with increments of 1
model = ProphecyPaperNetwork()
_act_handles, activation_signature = attach_relu_activation_hook(model)
_out_handles, layer_outputs = attach_layer_output_hook(model)


def satisfies_postcond(output: list, postcond: int):
  # if postcond is y1, then y1 must be greater than every other class. 
  postcond_var = output[postcond]
  result = True
  for index, val in enumerate(output):
    if index == postcond: continue
    result = result and (postcond_var > val)
  result = 1 if result == True else -1
  return result

def create_training_df(inputs, outputs, activation_signature, postcond: int):
  data = []
  for index, input_data in enumerate(inputs):    
    output = outputs[index].tolist()
    data_point = { 
      "input": input_data, 
      "output": output, 
      "satisfies_postcon": satisfies_postcond(output, postcond) 
    }
    
    for name, layer_activation in activation_signature.items():
      data_point[name] = layer_activation[index]
    data.append(data_point)
    
  return pd.DataFrame(data)

In [ ]:
# create input pairs for model
inputs = []
for x1 in np.arange(-50, 51, 1):
  for x2 in np.arange(-50, 51, 1):
    inputs.append([x1, x2])
    
_outputs = model(torch.tensor(inputs, dtype=torch.float32))
activation_signature = turn_bool_activation_to_int(activation_signature, to_list=True)

df = create_training_df(inputs, layer_outputs["final_output"], activation_signature, 0)
print(len(df))
df.head(10)

### Decision tree learning to find potential layer properties

In [ ]:
layer_name = "linear_relu_stack.1"
decision_tree = DecisionTree(df, X_col=layer_name, Y_col="satisfies_postcon")
leaves_with_activation_pattern = decision_tree.get_potential_layer_properties()
leaves_with_activation_pattern

In [ ]:
decision_tree.visualize_tree()

In [ ]:
dp = MarabouCoreDP()
result = []
for leaf_data in leaves_with_activation_pattern:
  decision_pattern = { layer_name: leaf_data["activation_pattern"] }
  print(decision_pattern)
  input_ranges = [[-100, 100], [-100, 100]]
  specification = [(np.array([[1, -1]]), np.array([0]))] # class = y1 (0)
  status, _, _, _ = dp.solve(decision_pattern, model, input_ranges, specification)
  result.append({"decision_pattern": decision_pattern, "status": status})
result

### Under-approximation box

In [ ]:
from algorithms.under_approximation_box import UnderApproximationBox

box = UnderApproximationBox()
input_property = {'linear_relu_stack.1': ['ON', 'OFF']}
model = ProphecyPaperNetwork()

In [ ]:
# get max and min of each input attributes from support
sat_postcon_df = df[df["satisfies_postcon"] == 1]
support_df = sat_postcon_df[sat_postcon_df['linear_relu_stack.1'].apply(lambda x: np.array_equal(x, [1, 0]))]
support_df.head(20)

In [ ]:
num_of_inputs = len(df.loc[0]["input"])

attr_max = []
attr_min = []

# attr_max = [50, -1] # for testing
# attr_min = [-49, -50] # for testing
for i in range(num_of_inputs):
  attr_max.append(support_df['input'].apply(lambda x: x[i]).max())
  attr_min.append(support_df['input'].apply(lambda x: x[i]).min())

print(attr_min)  
print(attr_max)

In [ ]:
problem, result = box.solve(input_property, attr_min, attr_max, model)

In [ ]:
import pulp
pulp.LpStatus[result]